<a href="https://colab.research.google.com/github/sujusudh/Complete-Python-3-Bootcamp/blob/master/3dunetchange2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import nibabel as nib
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers



In [ ]:
!pip install medpy
import medpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.3/156.3 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 MB 18.5 MB/s eta 0:00:00
  Created wheel for medpy: filename=MedPy-0.5.2-py3-none-any.whl size=224710 sha256=64e410bed347b97637c74e8be74b25b19bbc4e40428718f12b0dacbb388af03a
  Stored in directory: /root/.cache/pip/wheels/d4/33/ed/aaac5a347fb8d41679ca515b8f5c49dfdf49be15bdbb9a905d
Successfully built medpy


In [ ]:
from medpy.metric import binary

In [ ]:
# --- Define 3D U-Net ---
def build_unet_3d(input_shape):
    inputs = tf.keras.Input(input_shape)

    # Encoder
    c1 = layers.Conv3D(16, 3, activation='relu', padding='same')(inputs)
    c1 = layers.Dropout(0.1)(c1)
    c1 = layers.Conv3D(16, 3, activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling3D(2)(c1)

    c2 = layers.Conv3D(32, 3, activation='relu', padding='same')(p1)
    c2 = layers.Dropout(0.1)(c2)
    c2 = layers.Conv3D(32, 3, activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling3D(2)(c2)

    c3 = layers.Conv3D(64, 3, activation='relu', padding='same')(p2)
    c3 = layers.Dropout(0.2)(c3)
    c3 = layers.Conv3D(64, 3, activation='relu', padding='same')(c3)
    p3 = layers.MaxPooling3D(2)(c3)

    c4 = layers.Conv3D(128, 3, activation='relu', padding='same')(p3)
    c4 = layers.Dropout(0.2)(c4)
    c4 = layers.Conv3D(128, 3, activation='relu', padding='same')(c4)
    p4 = layers.MaxPooling3D(2)(c4)

    c5 = layers.Conv3D(256, 3, activation='relu', padding='same')(p4)
    c5 = layers.Dropout(0.3)(c5)
    c5 = layers.Conv3D(256, 3, activation='relu', padding='same')(c5)

    # Decoder
    u6 = layers.Conv3DTranspose(128, 2, strides=2, padding='same')(c5)
    u6 = layers.concatenate([u6, c4])
    c6 = layers.Conv3D(128, 3, activation='relu', padding='same')(u6)
    c6 = layers.Dropout(0.2)(c6)
    c6 = layers.Conv3D(128, 3, activation='relu', padding='same')(c6)

    u7 = layers.Conv3DTranspose(64, 2, strides=2, padding='same')(c6)
    u7 = layers.concatenate([u7, c3])
    c7 = layers.Conv3D(64, 3, activation='relu', padding='same')(u7)
    c7 = layers.Dropout(0.2)(c7)
    c7 = layers.Conv3D(64, 3, activation='relu', padding='same')(c7)

    u8 = layers.Conv3DTranspose(32, 2, strides=2, padding='same')(c7)
    u8 = layers.concatenate([u8, c2])
    c8 = layers.Conv3D(32, 3, activation='relu', padding='same')(u8)
    c8 = layers.Dropout(0.1)(c8)
    c8 = layers.Conv3D(32, 3, activation='relu', padding='same')(c8)

    u9 = layers.Conv3DTranspose(16, 2, strides=2, padding='same')(c8)
    u9 = layers.concatenate([u9, c1])
    c9 = layers.Conv3D(16, 3, activation='relu', padding='same')(u9)
    c9 = layers.Dropout(0.1)(c9)
    c9 = layers.Conv3D(16, 3, activation='relu', padding='same')(c9)

    outputs = layers.Conv3D(1, 1, activation='sigmoid')(c9)
    return models.Model(inputs, outputs)


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:

# --- Load MRI from subfolder ---
def load_mri_data(base_dir, patient_id, sequences=[ 'STIR', 'T2','PSIR','MP2RAGE']):
    patient_dir = os.path.join(base_dir, patient_id)
    volumes = []

    for seq in sequences:
      while True:
        for i in range(1:4):
          if(seq=='T2'&& 'STIR'):
            j=1
          elif(seq=='T2'&& 'PSIR'):
            j=2
          elif(seq=='T2'&& 'MP2RAGE'):
            j=3



        path = os.path.join(patient_dir, f'11-001_{seq}.nii.gz')
        if not os.path.exists(path):
            print(f'Missing: {path}')
            return None, None
        volumes.append(nib.load(path).get_fdata())

    mask_path = os.path.join(patient_dir, 'LESIONMASK.nii.gz')
    if os.path.exists(mask_path):
        mask = nib.load(mask_path).get_fdata()
    else:
        print(f'Missing mask: {mask_path}')
        return None, None

    stacked = np.stack(volumes, axis=-1)
    return stacked, mask


In [ ]:

# --- Preprocessing (resizing can be added if needed) ---
def preprocess_data(volume, mask, target_shape=(128, 128, 128)):
    volume = (volume - np.min(volume)) / (np.max(volume) - np.min(volume) + 1e-8)
    volume = np.expand_dims(volume, axis=0)
    mask = np.expand_dims(mask, axis=-1)
    mask = np.expand_dims(mask, axis=0)
    return volume.astype(np.float32), mask.astype(np.float32)


In [ ]:

# --- Load Dataset ---
def load_dataset(base_dir, patient_ids):
    X, y = [], []
    for pid in patient_ids:
        v, m = load_mri_data(base_dir, pid)
        if v is not None and m is not None:
            v, m = preprocess_data(v, m)
            X.append(v[0])
            y.append(m[0])
    return np.array(X), np.array(y)

In [ ]:

# --- Config ---
data_dir = "/content/drive/MyDrive/Colab Notebooks/mri1"
input_shape = (128, 128, 128, 4)
sequences = ['FLAIR', 'T2', 'STIR', 'PSIR','MP2RAGE']


In [ ]:

# --- List patient folders ---
patient_ids = sorted([name for name in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, name))])
train_ids = patient_ids[:int(0.8 * len(patient_ids))]
val_ids = patient_ids[int(0.8 * len(patient_ids)):]
print("Found patient folders:", patient_ids)

print("Training patients:", train_ids)
print("Validation patients:", val_ids)


Found patient folders: ['sub-001', 'sub-002']
Training patients: ['sub-001']
Validation patients: ['sub-002']


In [ ]:

# --- Build and compile model ---
model = build_unet_3d(input_shape)
model.compile(optimizer=optimizers.Adam(1e-4), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 128, 4)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d (Conv3D)     │ (None, 128, 128,  │      1,744 │ input_layer[0][0] │
│                     │ 128, 16)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 128, 128,  │          0 │ conv3d[0][0]      │
│                     │ 128, 16)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_1 (Conv3D)   │ (None, 128, 128,  │      6,928 │ dropout[0][0]     │
│                     │ 128, 16)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling3d       │ (None, 64, 64,    │          0 │ conv3d_1[0][0]    │
│ (MaxPooling3D)      │ 64, 16)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_2 (Conv3D)   │ (None, 64, 64,    │     13,856 │ max_pooling3d[0]… │
│                     │ 64, 32)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 64, 64,    │          0 │ conv3d_2[0][0]    │
│                     │ 64, 32)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_3 (Conv3D)   │ (None, 64, 64,    │     27,680 │ dropout_1[0][0]   │
│                     │ 64, 32)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling3d_1     │ (None, 32, 32,    │          0 │ conv3d_3[0][0]    │
│ (MaxPooling3D)      │ 32, 32)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_4 (Conv3D)   │ (None, 32, 32,    │     55,360 │ max_pooling3d_1[… │
│                     │ 32, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 32, 32,    │          0 │ conv3d_4[0][0]    │
│                     │ 32, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_5 (Conv3D)   │ (None, 32, 32,    │    110,656 │ dropout_2[0][0]   │
│                     │ 32, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling3d_2     │ (None, 16, 16,    │          0 │ conv3d_5[0][0]    │
│ (MaxPooling3D)      │ 16, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_6 (Conv3D)   │ (None, 16, 16,    │    221,312 │ max_pooling3d_2[… │
│                     │ 16, 128)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 16, 16,    │          0 │ conv3d_6[0][0]    │
│                     │ 16, 128)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_7 (Conv3D)   │ (None, 16, 16,    │    442,496 │ dropout_3[0][0]   │
│                     │ 16, 128)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling3d_3     │ (None, 8, 8, 8,   │          0 │ conv3d_7[0][0]  

 Total params: 5,646,209 (21.54 MB)

 Trainable params: 5,646,209 (21.54 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:

# --- Load data ---
X_train, y_train = load_dataset(data_dir, train_ids)
X_val, y_val = load_dataset(data_dir, val_ids)

print("Train shape:", X_train.shape)
print("Val shape:", X_val.shape)


Missing: /content/drive/MyDrive/Colab Notebooks/mri1/sub-001/STIR.nii.gz
Missing: /content/drive/MyDrive/Colab Notebooks/mri1/sub-002/STIR.nii.gz
Train shape: (0,)
Val shape: (0,)


In [ ]:
# --- Train ---
model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=1, epochs=10)


In [ ]:

# --- Dice Calculation ---
def calculate_dice(y_true, y_pred):
    y_pred = (y_pred.flatten() > 0.5).astype(np.uint8)
    y_true = y_true.flatten().astype(np.uint8)
    return binary.dc(y_pred, y_true)

dice_scores = []
for i in range(len(X_val)):
    pred = model.predict(np.expand_dims(X_val[i], 0))[0]
    dice = calculate_dice(y_val[i], pred)
    dice_scores.append(dice)

print(f"\nAverage Dice Coefficient on Validation Set: {np.mean(dice_scores):.4f}")
